In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [2]:
airport_city_file = open('训练集\\机场城市对应表.csv')
airport_city = pd.read_csv(airport_city_file)

weather_file = open('训练集\\2015年5月到2017年5月城市天气.csv', encoding='gbk')
weather = pd.read_csv(weather_file)

emergency_file = open('训练集\\2015年5月到2017年5月特情.csv', encoding='gbk')
emergency = pd.read_csv(emergency_file)

data_file = open('训练集\\2015年5月到2017年5月航班动态数据.csv', encoding='gbk')
data = pd.read_csv(data_file)

C:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
airport_city = airport_city.dropna()
airport_city_dict = {}
airport = airport_city['机场编码'].values
city = airport_city['城市名称'].values
for i in range(airport_city.shape[0]):
    airport_city_dict[airport[i]] = city[i]

In [4]:
emergency = emergency.drop('收集时间', axis = 1)
def emergency_clean(it):
    try:
        it = it.upper()
    except:
        return np.nan      
    if it in airport_city_dict.keys():
        return it
    else:
        return np.nan

emergency['特情机场'] = [emergency_clean(it) for it in emergency['特情机场']]
emergency = emergency.dropna()

emergency['开始时间'] = [datetime.strptime(it, "%Y-%m-%d %H:%M:%SZ") for it in emergency['开始时间']]
emergency['结束时间'] = [datetime.strptime(it, "%Y-%m-%d %H:%M:%SZ") for it in emergency['结束时间']]

In [5]:
data = data.drop(data.index[data['航班是否取消'] == '取消'], axis = 0)
data['y'] = (data['实际起飞时间'] - data['计划起飞时间']) > 3 * 60 * 60
data['延误时长'] = (data['实际起飞时间'] - data['计划起飞时间']) / (60 * 60)

new_cols = ['天气', '最低气温','最高气温','特情内容']  
data['计划起飞时间'] = [datetime.fromtimestamp(it) for it in data['计划起飞时间']]

data = data.loc[[airport in airport_city_dict.keys() for airport in data['出发机场']],::]
 
weather['日期'] = [datetime.strptime(it, "%Y/%m/%d").date() for it in weather['日期']]
weather = weather.loc[[city in airport_city_dict.values() for city in weather['城市']],::]

data['日期'] = [it.date() for it in data['计划起飞时间']]
data['城市'] = [airport_city_dict[it] for it in data['出发机场']]
data = data.drop(['航班是否取消'], axis = 1)

In [6]:
merge_data1 = pd.merge(data, weather, on = ['日期','城市'], how = 'left')

emergency['日期'] = [it.date() for it in emergency['开始时间']]
emergency['城市'] = [airport_city_dict[it] for it in emergency['特情机场']]

merge_data2 = pd.merge(merge_data1, emergency, on = ['日期','城市'], how = 'left')
merge_bool1 = merge_data2['开始时间'] <= merge_data2['计划起飞时间']
merge_bool2 = merge_data2['结束时间'] >= merge_data2['计划起飞时间']
merge_bool = np.stack([merge_bool1, merge_bool2], axis = 1)

merge_index = np.all(merge_bool, 1)
merge_data3_content = merge_data2['特情内容'].loc[merge_index]

merge_data2['特情内容'] = [np.nan for i in range(merge_data2.shape[0])]
merge_data2['特情内容'][merge_data3_content.index] = merge_data3_content

merge_data4_col = ['出发机场', '到达机场', '航班编号', '计划起飞时间', '计划到达时间', '实际起飞时间', '实际到达时间', '飞机编号',
       '延误时长', '日期', '城市', '天气', '最低气温', '最高气温', '特情内容', 'y']

merge_data4 = merge_data2.loc[::, merge_data4_col]
merge_data4.to_csv('merge_data.csv', index = False)

C:\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
